# Install Libraries

In [1]:
try:
  import torch
except:
  !pip install torch
  import torch

try:
  import transformers
except:
  !pip install transformers
  import transformers

try:
  import datasets
except:
  !pip install datasets
  import datasets

try:
  import sklearn
except:
  !pip install sklearn
  import sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

# Load Tokenizer

In [2]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#Load and preprocess data

In [3]:
from transformers import DataCollatorWithPadding

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

dataset = datasets.load_dataset("ag_news")

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["test"]

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

# Load Model

In [4]:
from transformers import AutoModelForSequenceClassification

num_labels = len(train_dataset.unique("labels"))
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Training

In [5]:
def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=1)
    return {"accuracy": sklearn.metrics.accuracy_score(labels, preds)}

training_args = transformers.TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    # learning_rate=2e-5,
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=8,
    # num_train_epochs=3,
    # weight_decay=0.01,
    report_to=["none"],
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.263500,0.246406,0.934605
2,0.167200,0.250971,0.944079
3,0.116100,0.272214,0.944605


TrainOutput(global_step=45000, training_loss=0.19093647622002496, metrics={'train_runtime': 4142.0906, 'train_samples_per_second': 86.913, 'train_steps_per_second': 10.864, 'total_flos': 7720216616669760.0, 'train_loss': 0.19093647622002496, 'epoch': 3.0})

# Model Evaluation and Predictions

In [6]:
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")


Accuracy: 0.9446052631578947


In [13]:
text = "The new Apple product is very good"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

for key, value in inputs.items():
    inputs[key] = value.to(model.device)

outputs = model(**inputs)
logits = outputs.logits
predicted_class = logits.argmax().item()

label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Science/Tech"
}

print(f"Input: \"{text}\", Predicted Class: {label_map[predicted_class]}")

Input: "The new Apple product is very good", Predicted Class: Science/Tech


# Save Model

In [14]:
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json',
 './fine_tuned_distilbert/tokenizer.json')

In [15]:
!git add fine_tuned_distilbert/

git version 2.34.1
